# Taylor videos on skeletons for action recognition

## 1. Taylor videos on skeletons

In [10]:
import scipy
from scipy import io
import torch

In [11]:
def factorial(n):
    """
    Calculate factorial of each element in the input tensor using recursion.
    """
    # Base case: factorial of 0 is 1
    if n == 0:
        return torch.tensor(1)
    else:
        return n * factorial(n - 1)

In [12]:
def taylor_video(videoinput, terms, temporal_block):
    
    # assume stride = 1
    video = torch.from_numpy(scipy.io.loadmat(videoinput)['skeletonsequence'])
    
    J, C, T = video.shape
    
    if temporal_block - 3 < terms:
        print('The given temporal block length is not enough to compute terms defined.')
        
    else:
        Taylor = torch.zeros((J, C, T - temporal_block + 1))
        
        for i in range(T - temporal_block + 1):
            # index starts from 0
            video_clip = video[:, :, i:i+temporal_block]
            slice_tensor = video[:, :, i].unsqueeze(2)
            dummy_clip = slice_tensor.repeat(1, 1, temporal_block)
            
            delta_temp = video_clip - dummy_clip
            
            D_temp = torch.zeros(J, C, terms + 2)
            
            temp = video_clip
            
            for j in range(terms+2):
                diff = temp[:, :, 1:] - temp[:, :, :-1]
                D_temp[:, :, j] = diff[:, :, 0]
                temp = diff
            
            M = torch.zeros((J, C, temporal_block))
            
            for order in range(terms+1):
                M = M + (D_temp[:, :, order].unsqueeze(-1) / factorial(order)) * torch.pow(delta_temp, order)               
                
            taylor_frame = M.sum(2)

            Taylor[:, :, i] = taylor_frame
    
    return Taylor

In [13]:
sk = '/Users/leiwang/Desktop/skeleton/a05_s01_e01_skeleton3D.txtssq.mat'

taylorskeleton = taylor_video(sk, 1, 4)

In [14]:
skeleton = scipy.io.loadmat(sk)['skeletonsequence']
print(skeleton.shape)
# print(type(skeleton))

skeleton = torch.from_numpy(skeleton)

(20, 3, 44)


In [15]:
taylorskeleton.shape

torch.Size([20, 3, 41])

## 2. Visualisation of original skeletons

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

num_frames = skeleton.shape[2]
num_joints = skeleton.shape[0]  

# Example connections of bones
sk_config = [[12, 10, 8, 1, 2, 0, 7, 9, 11], [19, 2, 3, 6, 5, 14, 16, 18], [6, 4, 13, 15, 17]]

# Function to plot 2D skeleton frame with connections (showing x as horizontal, y as vertical)
def plot_skeleton_frame_2d(frame_data, sk_config, ax):
    ax.clear()
    ax.set_title("2D Skeleton Frame (Horizontal X, Vertical Y)")

    # Plot joints (x as horizontal, y as vertical)
    for i in range(num_joints):
        x = frame_data[i, 0]
        y = frame_data[i, 1]
        ax.scatter(x, y, color='r')  # Plot x and y
    
    # Plot connections
    for connections in sk_config:
        for i in range(len(connections) - 1):
            start_joint = connections[i]
            end_joint = connections[i + 1]
            ax.plot([frame_data[start_joint, 0], frame_data[end_joint, 0]],
                    [frame_data[start_joint, 1], frame_data[end_joint, 1]], color='b')  # Plot connections
    
    # Set aspect ratio
    ax.set_aspect('equal')  # Equal aspect ratio

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Initialize the plot with the first frame
plot_skeleton_frame_2d(skeleton[:, :, 0], sk_config, ax)

# Function to update the plot for each frame
def update(frame):
    plot_skeleton_frame_2d(skeleton[:, :, frame], sk_config, ax)

# Create the animation
ani = FuncAnimation(fig, update, frames=num_frames, interval=100)  # Interval in milliseconds

# Disable xticks, yticks, xlabel, and ylabel
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('')

# Display the animation using HTML display
html_anim = HTML(ani.to_jshtml())
display(html_anim)

# Hide the ticks and axis labels
plt.close()  

## 3. Visualisation of Taylor videos on original skeletons

In [17]:
num_taylor = taylorskeleton.shape[2]
num_frames = skeleton.shape[2]
num_joints = skeleton.shape[0]

# Example connections of bones
sk_config = [[12, 10, 8, 1, 2, 0, 7, 9, 11], [19, 2, 3, 6, 5, 14, 16, 18], [6, 4, 13, 15, 17]]

# Function to plot 2D skeleton frame with connections (showing x as horizontal, y as vertical)
def plot_skeleton_frame_2d(frame_data, taylor_data, sk_config, ax):
    ax.clear()
    ax.set_title("2D Skeleton Frame (Horizontal X, Vertical Y)")

    # Plot joints (x as horizontal, y as vertical)
    for i in range(num_joints):
        x = frame_data[i, 0]
        y = frame_data[i, 1]
        a = taylor_data[i, 0]
        b = taylor_data[i, 1]
        ss = int(5000 * (abs(a) + abs(b)) / 2)  # Adjust the scaling factor as needed
        ax.scatter(x, y, color='r', s=ss)  # Plot x and y

    # Plot connections
    for connections in sk_config:
        for i in range(len(connections) - 1):
            start_joint = connections[i]
            end_joint = connections[i + 1]
            ax.plot([frame_data[start_joint, 0], frame_data[end_joint, 0]],
                    [frame_data[start_joint, 1], frame_data[end_joint, 1]], color='b')  # Plot connections

    # Set aspect ratio
    ax.set_aspect('equal')  # Equal aspect ratio

# Create a figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Initialize the plot with the first frame
plot_skeleton_frame_2d(skeleton[:, :, 0], taylorskeleton[:, :, 0], sk_config, ax)

# Function to update the plot for each frame
def update(frame):
    plot_skeleton_frame_2d(skeleton[:, :, frame], taylorskeleton[:, :, frame], sk_config, ax)

# Create the animation
ani = FuncAnimation(fig, update, frames=num_taylor, interval=100)  # Interval in milliseconds

# Display the animation using HTML display
html_anim = HTML(ani.to_jshtml())
display(html_anim)

# Hide the ticks and axis labels
plt.close()  # Close the plot to remove any residual ticks and labels


## 4. Compute Taylor videos on skeleton sequences

In [18]:
import os

# Directory path
directory = '/Users/leiwang/Desktop/skeleton'

# Loop through files in the directory
for filename in os.listdir(directory):
    # Check if the file ends with '.mat'
    if filename.endswith('.mat'):
        # Full path of the file
        filepath = os.path.join(directory, filename)
        # Process the file or perform operations as needed
        taylorskeleton = taylor_video(filepath, 1, 4)
        # New filename with 'taylor' added before '.mat'
        new_filename = filename.replace('.mat', 'taylor.mat')
        new_filepath = os.path.join(directory, new_filename)
        # print(new_filepath)
        print(f'Processing file: {filepath}')
        # Add your code here to work with the file
        taylorskeleton_array = taylorskeleton.double().numpy()

        # Save the NumPy array to a MATLAB file with key 'skeletonsequence'
        io.savemat(new_filepath, {'skeletonsequence': taylorskeleton_array}, appendmat=False, format='5')

Processing file: /Users/leiwang/Desktop/skeleton/a20_s03_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a17_s02_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a06_s05_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a13_s07_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a19_s08_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a10_s10_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a07_s08_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a11_s04_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a04_s06_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a09_s02_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a18_s05_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a15_s01_e01_skeleton3D.txtssq.mat
Proc

Processing file: /Users/leiwang/Desktop/skeleton/a08_s07_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a05_s03_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a15_s10_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a13_s04_e01_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a02_s08_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a01_s05_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a10_s02_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a03_s05_e02_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a08_s04_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a14_s07_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a09_s10_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a12_s10_e02_skeleton3D.txt

Processing file: /Users/leiwang/Desktop/skeleton/a12_s03_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a15_s08_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a02_s10_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a07_s01_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a19_s01_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a05_s02_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a10_s06_e02_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a02_s09_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a11_s07_e03_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a01_s07_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a08_s10_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a18_s03_e02_skeleton3D.txt

Processing file: /Users/leiwang/Desktop/skeleton/a17_s06_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a20_s07_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a03_s09_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a18_s01_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a04_s02_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a09_s06_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a01_s01_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a10_s06_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a19_s07_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a06_s01_e02_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a04_s10_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a13_s08_e01_skeleton3D.txtssq.ma

Processing file: /Users/leiwang/Desktop/skeleton/a14_s08_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a03_s10_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a09_s10_e03_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a04_s02_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a09_s06_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a18_s01_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a19_s08_e01_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a01_s05_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a20_s08_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a17_s09_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a06_s05_e01_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a08_s04_e03_skeleton

Processing file: /Users/leiwang/Desktop/skeleton/a11_s06_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a10_s02_e03_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a18_s07_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a12_s08_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a05_s10_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a06_s07_e02_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a13_s05_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a02_s02_e01_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a04_s01_e01_skeleton3D.txtssqtaylor.mat
Processing file: /Users/leiwang/Desktop/skeleton/a02_s10_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a15_s08_e03_skeleton3D.txtssq.mat
Processing file: /Users/leiwang/Desktop/skeleton/a01_s05_e03_skeleton3D.txt

Processing file: /Users/leiwang/Desktop/skeleton/a20_s01_e02_skeleton3D.txtssq.mat
